## Импорт библиотек

In [45]:
!pip install sentence_transformers
!pip install faiss-cpu --no-cache

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re
import string
import faiss
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

from sentence_transformers import SentenceTransformer, util
from typing import List
from tqdm import tqdm
from math import log

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка данных

In [47]:
try:
    marketing_productdealerkey = pd.read_csv('data/marketing_productdealerkey.csv', sep=';', quotechar='"', index_col ='id')
    marketing_product = pd.read_csv('data/marketing_product.csv', sep=';', index_col=0)
    marketing_dealerprice = pd.read_csv('data/marketing_dealerprice.csv', sep=';', index_col ='id')
    marketing_dealer = pd.read_csv('data/marketing_dealer.csv', sep=';')
except:
    from google.colab import drive
    drive.mount('/content/drive')
    DIR = '/content/drive/MyDrive/Projects'
    marketing_productdealerkey = pd.read_csv(f'{DIR}/Prosept/datasets/marketing_productdealerkey.csv', sep=';', quotechar='"', index_col ='id')
    marketing_product = pd.read_csv(f'{DIR}/Prosept/datasets/marketing_product.csv', sep=';', index_col=0)
    marketing_dealerprice = pd.read_csv(f'{DIR}/Prosept/datasets/marketing_dealerprice.csv', sep=';', index_col ='id')
    marketing_dealer = pd.read_csv(f'{DIR}/Prosept/datasets/marketing_dealer.csv', sep=';')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Обзор данных

In [48]:
# Функция для отображения первых 10 строк данных и получения общей информации о DataFrame
def display_and_info(df):
    display(df.head(10))
    print(df.info())
    print(df.isna().sum())
    print('Количество дубликатов:', df.duplicated().sum())

### marketing_dealerprice (результат работы парсера площадок дилеров)

*	product_key - уникальный номер позиции;
*	price - цена;
*	product_url - адрес страницы, откуда собраны данные;
*	product_name - заголовок продаваемого товара;
*	date - дата получения информации;
*	dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)


In [49]:
display_and_info(marketing_dealerprice)

,product_key,price,product_url,product_name,date,dealer_id
id,,,,,,
2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2
7,831859,370.0,https://akson.ru//p/propitka_prosept_aquaisol_...,"Пропитка PROSEPT Aquaisol для камня, концентра...",2023-07-11,2
8,546406,235.0,https://akson.ru//p/sredstvo_dlya_mytya_plitki...,Средство для мытья плитки и керамогранита Pros...,2023-07-11,2
9,831858,1648.0,https://akson.ru//p/propitka_prosept_aquaisol_...,"Пропитка PROSEPT Aquaisol для камня, концентра...",2023-07-11,2
10,857015,371.0,https://akson.ru//p/shpatlevka_zamazka_prosept...,Шпаклевка выравнивающая акриловая PROSEPT Plas...,2023-07-11,2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20416 entries, 2 to 20570
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   20416 non-null  object 
 1   price         20416 non-null  float64
 2   product_url   20182 non-null  object 
 3   product_name  20416 non-null  object 
 4   date          20416 non-null  object 
 5   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ MB
None
product_key       0
price             0
product_url     234
product_name      0
date              0
dealer_id         0
dtype: int64
Количество дубликатов: 726


In [50]:
# Преобразование столбца 'date' в тип данных datetime

marketing_dealerprice['date'] = pd.to_datetime(marketing_dealerprice['date'])

* В качестве запроса для модели будет использоваться колонка product_name.
* У дилера 6 названия без пробелов.
* Присутствуют дубликаты.

### marketing_dealer (список дилеров)

In [51]:
display_and_info(marketing_dealer)

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora
5,7,Komus
6,9,Megastroy
7,10,OnlineTrade
8,11,Petrovich
9,12,sdvor


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18 non-null     int64 
 1   name    18 non-null     object
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes
None
id      0
name    0
dtype: int64
Количество дубликатов: 0


### marketing_productdealerkey (таблица матчинга товаров заказчика и товаров дилеров)

* key - внешний ключ к marketing_dealerprice
*	product_id - внешний ключ к marketing_product
*	dealer_id - внешний ключ к marketing_dealer


In [52]:
display_and_info(marketing_productdealerkey)

,key,dealer_id,product_id
id,,,
1,546227,2,12
2,651265,2,106
3,546257,2,200
4,546408,2,38
5,651258,2,403
6,546234,2,18
7,546355,2,39
8,831859,2,396
9,832159,2,284


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1700 entries, 1 to 2023
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key         1700 non-null   object
 1   dealer_id   1700 non-null   int64 
 2   product_id  1700 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 53.1+ KB
None
key           0
dealer_id     0
product_id    0
dtype: int64
Количество дубликатов: 0


* у дилера с id=6 вместо номеров ключей вставлены url
* разное количество продуктов у дилера по сравнению с таблицей `marketing_dealerprice`

### marketing_product (список товаров, которые производит и распространяет заказчик)

*	article - артикул товара;
*	ean_13 - код товара (см. EAN 13)
*	name - название товара;
*	cost - стоимость;
*	min_recommended_price - рекомендованная минимальная цена;
*	recommended_price - рекомендованная цена;
*	category_id - категория товара;
*	ozon_name - названиет товара на Озоне;
*	name_1c - название товара в 1C;
*	ozon_article - описание для Озон;
*	wb_name - название товара на Wildberries;
*	ym_article - артикул для Яндекс.Маркета;
*	wb_article - артикул для Wildberries;

In [53]:
display_and_info(marketing_product)

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN
5,220,051-6,4.610093e+12,Грунт БЕТОНКОНТАКТготовый состав / 6 кг,703.00,1339.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",453193200.0,149699633.0,051-6,NaN
6,385,051-10,4.680008e+12,Грунт БЕТОНКОНТАКТготовый состав / 12 кг,1111.00,2505.0,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",189522783.0,149699627.0,051-10,NaN
7,114,125-5,4.680008e+12,"Средство для удаления технических масел, смазо...",711.82,1663.0,35.0,"Средство для удаления технических масел, смазо...","Средство для удаления технических масел, смазо...","Средство для удаления технических масел, смазо...",189522869.0,149992148.0,125-5,NaN
8,505,СФ001-5,4.610093e+12,Антисептик универсальный суперсильный,200.00,510.0,20.0,Антисептик универсальный суперсильный,Антисептик универсальный суперсильный,Антисептик универсальный суперсильный,NaN,NaN,NaN,NaN
9,32,100-1,4.680008e+12,Средство для мытья светлых полов с отбеливающи...,132.72,311.0,40.0,Профессиональное средство для мытья светлых по...,Профессиональное средство для мытья светлых по...,Профессиональное средство для мытья светлых по...,451088021.0,149705841.0,100-1,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 495
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   article            496 non-null    object 
 2   ean_13             464 non-null    float64
 3   name               494 non-null    object 
 4   cost               491 non-null    float64
 5   recommended_price  491 non-null    float64
 6   category_id        447 non-null    float64
 7   ozon_name          458 non-null    object 
 8   name_1c            485 non-null    object 
 9   wb_name            455 non-null    object 
 10  ozon_article       365 non-null    float64
 11  wb_article         340 non-null    float64
 12  ym_article         337 non-null    object 
 13  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(1), object(7)
memory usage: 58.1+ KB
None
id                     0
article                0
ean_13   

* Присутствуют пропуски.
* В ключевом столбце name есть 2 пропуска.
* В остальных столбцах тоже есть пропуски: можно их заполнить на 'unknown'.
* Названия продуктов написаны с ошибкой: нет пробелов между латиницей и кириллицей.

In [54]:
# Рассмотрим строки с пропусками
marketing_product[marketing_product.name.isna()]

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
23,503,0024-7 о,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150126213.0,NaN,NaN
35,504,w022-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Можем удалить эти строки, так как нет вообще никакой информации.



In [55]:
# Удаление строк, в которых значение в столбце 'name' является пропущенным (NaN)
marketing_product.dropna(subset=['name'], inplace=True)

In [56]:
# Удаление строк, в которых значение в столбце 'name' равно пустой строке
marketing_product = marketing_product[marketing_product.name != '   ']

In [57]:
# Замена пропущенных значений в DataFrame 'marketing_product' строкой 'unknown'
marketing_product.fillna('unknown', inplace=True)

В качестве корпуса названий для обучения можем использовать:
* только название от производителя
* объединить название от производителя 'name' и у других дилеров 'ozon_name', 'name_1c', 'wb_name'

## Предобработка текста

### Колонка marketingproduct.name

In [58]:
product_name = marketing_product.name
# Переводим текст в нижний регистр
product_name = product_name.apply(lambda x: x.lower())
# Удаление знаков препинания
product_name = product_name.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation+'«»–'), ' ', x))
# Разделение на латиницу и кириллицу
product_name = product_name.apply(lambda x: ' '.join(re.split(r'([a-zA-Z]+|[a-zA-Z]+)', x)))
# Удаление лишних пробелов, замена ошибочных фрагментов и удаление слов с числами в product_name
product_name = product_name.apply(lambda x: ' '.join(x.split()))
product_name = product_name.apply(lambda x: x.replace(' редство', ' средство').replace('c ', ''))
product_name = product_name.apply(lambda x: re.sub('\w*\d\w*', '', x))

In [59]:
# Расширение списка стоп-слов
stop_words.extend(['что', 'это', 'так',
                    'вот', 'быть', 'как',
                    'в', 'к', 'за', 'из', 'из-за', 'с',
                    'на', 'ок', 'кстати',
                    'который', 'мочь', 'весь',
                    'еще', 'также', 'свой',
                    'ещё', 'самый', 'ул', 'главные', 'играет',
                    'и','y', 'c', 'для', 'prosept', 'просепт',
                    'для', 'средство' , 'кг', 'г', 'мл', 'л', 'шт' ])

In [60]:
# Функция для удаления стоп-слов из текста
def remove_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [61]:
# Применим функцию удаления стоп-слов к столбцу product_name
product_name = product_name.apply(lambda x: remove_stopwords(x))
product_name

0               антисептик невымываемый ultra концентрат
1                             антигололед готовый состав
2                      герметик акриловый цвет сосна ф п
3      кондиционер белья ароматом королевского ириса ...
4                          герметик акриловой цвет белый
                             ...                        
491    уборки помещений пожара дезинфицирующим эффект...
492    жидкое моющее стирки шерсти шелка деликатных т...
493    чистки гриля духовых шкафов cooky grill gel ко...
494    мытья полов полимерным покрытием multipower br...
495    усиленного действия удаления ржавчины минераль...
Name: name, Length: 493, dtype: object

### Колонка marketing_dealerprice.dealer_name

In [62]:
dealer_name = marketing_dealerprice.product_name
# нижний регистр
dealer_name = dealer_name.apply(lambda x: x.lower())
# убираем знаки препинания
dealer_name = dealer_name.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation+'«»–'), ' ', x))
dealer_name = dealer_name.apply(lambda x: re.sub('\s+\d+', ' ', x))

# лишние пробелы
dealer_name = dealer_name.apply(lambda x: ' '.join(x.split()))
dealer_name = dealer_name.apply(lambda x: remove_stopwords(x))
dealer_name = dealer_name.apply(lambda x: ' '.join(x.split()))

In [63]:
dealer_name[[14541, 14513, 15972, 15944]]

id
14541    антижук
14513    плесени
15972    антижук
15944    плесени
Name: product_name, dtype: object

In [64]:
# Построение словаря стоимости, исходя из закон Ципфа
words = open('/content/drive/MyDrive/Projects/Prosept/files/new_words.txt').read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
nums = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for n in nums:
    wordcost[n] = log(2)
maxword = max(len(x) for x in words)

In [65]:
# Функция  для предсказания расположения пробелов в строке без пробелов
def infer_spaces(s):

    # Функция возвращает пару (match_cost, match_length)
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Построение массива стоимости
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Восстановление строки с минимальной стоимостью
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [66]:
# создание new_df, который содержит только  строки, где длина строки в столбце dealer_name менее двух слов.
new_df = dealer_name[dealer_name.apply(lambda x:len(x.split())<2)]
new_df

id
393                             плесени
420                      антижукprosept
421         антисептикproseptecoultra5л
422           антисептикproseptecoultra
423      антисептикproseptecoultraкор5л
                      ...              
15944                           плесени
17702                           антижук
17673                           плесени
19268                           антижук
19241                           плесени
Name: product_name, Length: 184, dtype: object

In [67]:
# Применяем функцию только к нужным строкам
for i in new_df.index:
    string = dealer_name.loc[i]
    dealer_name.loc[i] = infer_spaces(string)

In [68]:
dealer_name[new_df.index]

id
393                                  плесени
420                          антижук prosept
421          антисептик prosept eco ultra 5л
422             антисептик prosept eco ultra
423      антисептик prosept eco ultra кор 5л
                        ...                 
15944                                плесени
17702                                антижук
17673                                плесени
19268                                антижук
19241                                плесени
Name: product_name, Length: 184, dtype: object

In [69]:
# Применяем функцию remove_stopwords ко всем значениям в столбце dealer_name
dealer_name = dealer_name.apply(lambda x: remove_stopwords(x))
dealer_name[516]

'лакдлякамняprosept053 полуматовый'

In [70]:
# Удаляем последовательности цифр, начинающиеся с пробела, из каждого значения в столбце dealer_name
dealer_name = dealer_name.apply(lambda x: re.sub('\s+\d\w*', '', x))
dealer_name

id
2                            универсальное universal spray
3                 концентрат multipower мытья полов цитрус
4                         чистки люстр universal anti dust
5                          удалитель ржавчины rust remover
6                        моющее бани сауны multipower wood
                               ...                        
20566    огнебиозащита древесины группа красный готовый...
20567              антисептик многофункциональный фбс гост
20568                                    удаления ржавчины
20569    герметик акриловый межшовный деревянных констр...
20570    краска грунт фасадная плит osb proff liquid ru...
Name: product_name, Length: 20416, dtype: object

In [71]:
# Убираем лишние пробелы в каждом значении столбца dealer_name
dealer_name = dealer_name.apply(lambda x: ' '.join(x.split()))
dealer_name

id
2                            универсальное universal spray
3                 концентрат multipower мытья полов цитрус
4                         чистки люстр universal anti dust
5                          удалитель ржавчины rust remover
6                        моющее бани сауны multipower wood
                               ...                        
20566    огнебиозащита древесины группа красный готовый...
20567              антисептик многофункциональный фбс гост
20568                                    удаления ржавчины
20569    герметик акриловый межшовный деревянных констр...
20570    краска грунт фасадная плит osb proff liquid ru...
Name: product_name, Length: 20416, dtype: object

## Эмббединги

In [72]:
from sentence_transformers import SentenceTransformer, util

In [73]:
# Загружаем предварительно обученную модель SentenceTransformer LaBSE
model = SentenceTransformer('sentence-transformers/LaBSE')

In [74]:
# Получение векторных представлений для наименований товаров
corpus_embeddings = model.encode(product_name.values, convert_to_tensor=True)
corpus_embeddings

tensor([[-0.0032,  0.0057,  0.0093,  ..., -0.0007, -0.0031, -0.0560],
        [ 0.0137, -0.0172,  0.0459,  ...,  0.0245, -0.0400, -0.0686],
        [ 0.0108, -0.0338,  0.0310,  ...,  0.0234, -0.0147, -0.0107],
        ...,
        [ 0.0632, -0.0680, -0.0642,  ..., -0.0078,  0.0088, -0.0261],
        [-0.0019,  0.0185,  0.0328,  ..., -0.0246,  0.0175, -0.0368],
        [ 0.0027, -0.0097,  0.0122,  ..., -0.0026,  0.0352, -0.0238]])

In [75]:
# Получение векторного представления запроса с использованием модели LaBSE
query = dealer_name[2679]
query_embedding = model.encode(query, convert_to_tensor=True)

In [76]:
# Используем cosine-similarity и torch.topk для поиска пяти наилучших результатов
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=5)

# Выводим запрос и пять наиболее похожих предложений
print("\n======================\n")
print("Запрос:", query)
print("\nТоп 5 наиболее похожих предложений в корпусе:")

# Извлекаем индексы и оценки лучших результатов
best_idx = []
for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy()
    idx = idx.cpu().data.numpy()
    best_idx.append(idx)

# Выводим индексы лучших результатов
print(best_idx)

# Извлекаем и отображаем рекомендуемые строки из marketing_product
recommendations = marketing_product.iloc[best_idx]
recommendations



Запрос: устранения засоров bath prof концентрат

Топ 5 наиболее похожих предложений в корпусе:
[array(130), array(474), array(326), array(443), array(432)]


,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
133,59,111-075,4680008146786.0,Средство для устранения засоров в трубахBath P...,76.78,180.0,45.0,Средство для устранения засоров в трубахBath P...,Средство для устранения засоров в трубахBath P...,Средство для устранения засоров в трубахBath P...,unknown,unknown,unknown,unknown
477,61,111-5,4680008141354.0,Средство для устранения засоров в трубахBath P...,409.0,956.0,45.0,Средство для прочистки труб от засоров PROSEPT...,Средство для прочистки труб от засоров PROSEPT...,Средство для прочистки труб от засоров PROSEPT...,411379449.0,150033031.0,111-5,111-50
329,60,111-1,4680008141644.0,Средство для устранения засоров в трубахBath P...,102.0,239.0,45.0,Средство для прочистки труб от засоров PROSEPT...,Средство для прочистки труб от засоров PROSEPT...,Средство для прочистки труб от засоров PROSEPT...,411379450.0,150033032.0,111-1,111-10
446,5,104-5,4680008141767.0,Универсальное моющее средствоUniversal Prof к...,601.76,1405.0,56.0,Профессиональное универсальное моющее средство...,Профессиональное универсальное моющее средство...,Профессиональное универсальное моющее средство...,449915356.0,149811051.0,104-5,unknown
435,4,104-1,4680008141750.0,Универсальное моющее средствоUniversal Prof к...,150.0,352.0,56.0,Профессиональное универсальное моющее средство...,Профессиональное универсальное моющее средство...,Профессиональное универсальное моющее средство...,449915357.0,149811058.0,104-1,unknown


## Метрика

In [77]:
# Функция для вычисления точности для топ-k рекомендаций
def accuracy_k(scores: List[int]) -> float:
     return round(sum(scores) / len(scores), 2)

In [78]:
from typing import List

In [79]:
# Функция get_recommendations для получения рекомендаций
def get_recommendations(model, corpus_embeddings, dealer_names, dealer_product_key, k=3) -> List[int]:

    query = dealer_names[dealer_product_key]
    query_embedding = model.encode(query, convert_to_tensor=True)

    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=k)

    best_idx = []

    for score, idx in zip(top_results[0], top_results[1]):
        score = score.cpu().data.numpy()
        idx = idx.cpu().data.numpy()
        best_idx.append(idx)

    return best_idx

In [80]:
# Функция recommendations_for_metric для оценки точности рекомендаций
def recommendations_for_metric(model, dealer_names, corpus_embeddings, k=3) -> List[bool]:

    scores = []
    false = []
    empty_result = []
    for idx in tqdm(dealer_names.index):

        product_key = marketing_dealerprice.product_key[idx]
        dealer_id = marketing_dealerprice.dealer_id[idx]
        true_id = marketing_productdealerkey[(marketing_productdealerkey.dealer_id == dealer_id) & (marketing_productdealerkey.key == product_key)]['product_id'].values
        if not true_id:
            empty_result.append(idx)
        else:
            best_idx = get_recommendations(model, corpus_embeddings, dealer_names, idx, k)

            recom_product_id = [x for x in marketing_product.id.iloc[best_idx]]

            score = any(i == true_id for i in recom_product_id)
            if not score:
                false.append(idx)
            scores.append(score)

    return empty_result, false, scores

In [81]:
empty_result, false, scores = recommendations_for_metric(model, dealer_name, corpus_embeddings, k=5)

  3%|▎         | 513/20416 [01:07<59:35,  5.57it/s]<ipython-input-80-c9a29b773efa>:12: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not true_id:
100%|██████████| 20416/20416 [37:55<00:00,  8.97it/s]


In [82]:
# Рассмотрим строки в которых получили false
dealer_name[false]

id
10         шпаклевка выравнивающая акриловая plastix белая
13       антисептик eco ultra невымываемый коричневый г...
19       огнебиозащита prof группа наружных внутренних ...
27       антисептик eco ultra невымываемый коричневый г...
44       удаления ржавчины минеральных отложений bath a...
                               ...                        
20511    герметик акриловый межшовный деревянных констр...
20520    герметик акриловый межшовный деревянных констр...
20528    герметик акриловый межшовный деревянных констр...
20536                 строительный антисептик невымываемый
20537    герметик акриловый межшовный деревянных констр...
Name: product_name, Length: 2237, dtype: object

In [83]:
print(marketing_dealerprice.product_name.loc[27])
print(dealer_name[27])

Антисептик PROSEPT Eco Ultra невымываемый, коричневый, готовый состав 10л
антисептик eco ultra невымываемый коричневый готовый состав


In [84]:
# Применим функцию get_recommendations чтобы получить рекомендации
best_idx = get_recommendations(model, corpus_embeddings, dealer_name, 27, k=3)
# recommendations = marketing_product.iloc[best_idx]
# recommendations

In [85]:
print(best_idx)

[array(342), array(334), array(139)]


In [86]:
best_idx[0]

array(342)

In [87]:
lambda_best_idx = lambda i:int(best_idx[i])

In [89]:
print(best_idx)

[array(342), array(334), array(139)]


In [90]:
best_idx[0] = int(best_idx[0])
best_idx

[342, array(334), array(139)]

In [91]:
# Для заданного product_key и dealer_id находим соответствующий product_id в marketing_productdealerkey
product_key = marketing_dealerprice.product_key[27]
dealer_id = marketing_dealerprice.dealer_id[27]
true_id = marketing_productdealerkey[(marketing_productdealerkey.dealer_id==dealer_id) & (marketing_productdealerkey.key==product_key)]['product_id'].values
true_id

array([276])

In [92]:
marketing_product[marketing_product.id==276]

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
191,276,062-10,4680008148995.0,"Антисептик ECO ULTRA, коричневый / 10 л",476.0,1136.0,20.0,"Антисептик невымываемый PROSEPT ECO ULTRA, 10 л.","Антисептик ECO ULTRA, коричневый / 10 л","Антисептик невымываемый PROSEPT ECO ULTRA, 10 л.",253565305.0,150033477.0,062-10,unknown


In [93]:
# Рассчет точности с помощью accuracy_k
accuracy_k(scores)

0.87

In [94]:
# Посмотри длину списка индексов, для которых не удалось найти соответствующий product_id
len(empty_result)

2849

In [95]:
# Выборка product_key для строк с пустыми значениями
empty_product_key = list(marketing_dealerprice.product_key.loc[empty_result])

In [96]:
# Фильтрация строк, где значение столбца 'key' содержится в списке empty_product_key
marketing_productdealerkey[marketing_productdealerkey.key.apply(lambda x: x in empty_product_key)]

,key,dealer_id,product_id
id,,,


In [97]:
# Вызываем функцию recommendations_for_metric, чтобы получить рекомендации и оценку точности
empty_result, false, scores = recommendations_for_metric(model, dealer_name, corpus_embeddings, k=7)

  3%|▎         | 513/20416 [01:08<44:43,  7.42it/s]<ipython-input-80-c9a29b773efa>:12: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not true_id:
100%|██████████| 20416/20416 [38:25<00:00,  8.85it/s]


In [98]:
accuracy_k(scores)

0.91

## Faiss

In [99]:
# Функция get_recommendations_faiss для эффективного поиска наилучших соответствий векторов эмбеддингов
def get_recommendations_faiss(model, corpus_embeddings, dealer_names, dealer_product_key, k=3) -> List[int]:

    # Получение размерности вектора
    vector_dimension = corpus_embeddings.shape[1]

    # Создание Faiss индекса
    index = faiss.IndexFlatL2(vector_dimension)

    # Нормализация векторов в корпусе
    faiss.normalize_L2(corpus_embeddings.numpy())

    # Добавление векторов в индекс
    index.add(corpus_embeddings)

    # Получение эмбеддинга запроса и его нормализация
    query = dealer_names[dealer_product_key]
    query_embedding = model.encode(query, convert_to_tensor=True).numpy()
    faiss.normalize_L2(query_embedding)

    # Поиск наилучших результатов с использованием Faiss
    top_results = index.search(query_embedding, k=k)

    best_idx = []

    for score, idx in zip(top_results[0], top_results[1]):
        score = score.cpu().data.numpy()
        idx = idx.cpu().data.numpy()
        best_idx.append(idx)

    return best_idx

In [100]:
# Вызываем функцию recommendations_for_metric, чтобы получить рекомендации и оценку точности
empty_result_faiss, false_faiss, scores_faiss = recommendations_for_metric(model, dealer_name, corpus_embeddings, k=5)







  3%|▎         | 513/20416 [01:08<37:45,  8.79it/s]<ipython-input-80-c9a29b773efa>:12: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not true_id:
100%|██████████| 20416/20416 [38:45<00:00,  8.78it/s]


In [101]:
accuracy_k(scores_faiss)

0.87

## Feature engineering

### product_corpus

In [103]:
product_name = marketing_product.name
# нижний регистр
product_name = product_name.apply(lambda x: x.lower())
# убираем знаки препинания
#product_name = product_name.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation + '«»–'), ' ', x))
# разделяем латиницу и кириллицу
product_name = product_name.apply(lambda x: ' '.join(re.split(r'([a-zA-Z]+|[a-zA-Z]+)', x)))
# лишние пробелы
product_name = product_name.apply(lambda x: ' '.join(x.split()))
product_name = product_name.apply(lambda x: x.replace(' редство', ' средство').replace('c ', ''))

In [104]:
# Расширение списка стоп-слов
stop_words.extend(['что', 'это', 'так',
                   'вот', 'быть', 'как',
                   'в', 'к', 'за', 'из', 'из-за', 'с',
                   'на', 'ок', 'кстати',
                   'который', 'мочь', 'весь',
                   'еще', 'также', 'свой',
                   'ещё', 'самый', 'ул', 'главные', 'играет', 'и', 'y', 'c', 'для', 'prosept', 'просепт', 'для',
                   'средство', 'кг', 'г', 'мл', 'л', 'шт'])

In [105]:
product_name

0      антисептик невымываемый prosept ultra концентр...
1        антигололед - 32 prosept готовый состав / 12 кг
2               герметик акриловый цвет сосна, ф/п 600мл
3      кондиционер для белья с ароматом королевского ...
4                    герметик акриловой цвет белый, 7 кг
                             ...                        
491    средство для уборки помещений после пожара с д...
492    жидкое моющее средство для стирки шерсти, шелк...
493    средство для чистки гриля и духовых шкафов coo...
494    средство для мытья полов с полимерным покрытие...
495    средство усиленного действия для удаления ржав...
Name: name, Length: 493, dtype: object

In [106]:
# Извлечение информации о объеме или количестве продукта из наименования и преобразование в единый формат
product_volume = product_name.apply(lambda x: (re.findall(r'\s*\d+\s*(?:л|г|мл|кг|шт|штук)', x)))
product_volume = product_volume.apply(lambda x: "".join(x).replace(" ", ""))
product_volume

0         1л
1       12кг
2      600мл
3         2л
4        7кг
       ...  
491       5л
492       1л
493       5л
494       5л
495      75л
Name: name, Length: 493, dtype: object

In [107]:
# Функция remove_stopwords для удаления стоп-слова из текста
def remove_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text


product_name = product_name.apply(lambda x: remove_stopwords(x))
product_name

0      антисептик невымываемый ultra концентрат 1:10 / 1
1                   антигололед - 32 готовый состав / 12
2               герметик акриловый цвет сосна, ф/п 600мл
3      кондиционер белья ароматом королевского ириса ...
4                       герметик акриловой цвет белый, 7
                             ...                        
491    уборки помещений пожара дезинфицирующим эффект...
492    жидкое моющее стирки шерсти, шелка деликатных ...
493    чистки гриля духовых шкафов cooky grill gel ко...
494    мытья полов полимерным покрытием multipower br...
495    усиленного действия удаления ржавчины минераль...
Name: name, Length: 493, dtype: object

In [108]:
# Удаляем слова, содержащих цифры, в столбце product_name
product_name = product_name.apply(lambda x: re.sub('\w*\d\w*', '', x))
product_name

0          антисептик невымываемый ultra концентрат : / 
1                       антигололед -  готовый состав / 
2                    герметик акриловый цвет сосна, ф/п 
3      кондиционер белья ароматом королевского ириса ...
4                        герметик акриловой цвет белый, 
                             ...                        
491    уборки помещений пожара дезинфицирующим эффект...
492    жидкое моющее стирки шерсти, шелка деликатных ...
493    чистки гриля духовых шкафов cooky grill gel ко...
494    мытья полов полимерным покрытием multipower br...
495    усиленного действия удаления ржавчины минераль...
Name: name, Length: 493, dtype: object

In [109]:
# Удаляем лишние пробелы в каждом значении столбца product_name
product_name = product_name.apply(lambda x: ' '.join(x.split()))
product_name

0           антисептик невымываемый ultra концентрат : /
1                         антигололед - готовый состав /
2                     герметик акриловый цвет сосна, ф/п
3      кондиционер белья ароматом королевского ириса ...
4                         герметик акриловой цвет белый,
                             ...                        
491    уборки помещений пожара дезинфицирующим эффект...
492    жидкое моющее стирки шерсти, шелка деликатных ...
493    чистки гриля духовых шкафов cooky grill gel ко...
494    мытья полов полимерным покрытием multipower br...
495    усиленного действия удаления ржавчины минераль...
Name: name, Length: 493, dtype: object

In [110]:
# Создание корпуса путем объединения названия продуктов и их объемы
product_corpus = product_name + ' ' + product_volume
product_corpus

0        антисептик невымываемый ultra концентрат : / 1л
1                    антигололед - готовый состав / 12кг
2               герметик акриловый цвет сосна, ф/п 600мл
3      кондиционер белья ароматом королевского ириса ...
4                     герметик акриловой цвет белый, 7кг
                             ...                        
491    уборки помещений пожара дезинфицирующим эффект...
492    жидкое моющее стирки шерсти, шелка деликатных ...
493    чистки гриля духовых шкафов cooky grill gel ко...
494    мытья полов полимерным покрытием multipower br...
495    усиленного действия удаления ржавчины минераль...
Name: name, Length: 493, dtype: object

### dealer_corpus

In [112]:
dealer_name = marketing_dealerprice.product_name
# нижний регистр
dealer_name = dealer_name.apply(lambda x: x.lower())
# убираем знаки препинания
#dealer_name = dealer_name.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation+'«»–'), '', x))
dealer_name

id
2        средство универсальное prosept universal spray...
3        концентрат prosept multipower для мытья полов,...
4        средство для чистки люстр prosept universal an...
5        удалитель ржавчины prosept rust remover 0,5л 0...
6        средство моющее для бани и сауны prosept multi...
                               ...                        
20566    огнебиозащита для древесины prosept 2 группа, ...
20567         антисептик многофункциональный фбс, гост 5 л
20568         средство для удаления ржавчины prosept, 1 шт
20569    герметик акриловый межшовный для деревянных ко...
20570    краска-грунт фасадная для плит osb proff 3 в 1...
Name: product_name, Length: 20416, dtype: object

In [113]:
dealer_volume = dealer_name.apply(lambda x: (re.findall(r'\s*\d+\s*(?:л|г|мл|кг|шт|штук)', x)))
dealer_volume = dealer_volume.apply(lambda x: "".join(x).replace(" ", ""))
dealer_volume

id
2        500мл
3           1л
4        500мл
5           5л
6           1л
         ...  
20566    2г10л
20567       5л
20568      1шт
20569      6кг
20570      7кг
Name: product_name, Length: 20416, dtype: object

In [114]:
# создание new_df, который содержит только  строки, где длина строки в столбце dealer_name менее двух слов
new_df = dealer_name[dealer_name.apply(lambda x: len(x.split()) < 2)]

In [115]:
# Применяем функцию только к нужным строкам
for i in new_df.index:
    string = dealer_name.loc[i]
    dealer_name.loc[i] = infer_spaces(string)

In [116]:
# Убираем лишние пробелы в каждом значении столбца dealer_name
dealer_name = dealer_name.apply(lambda x: ' '.join(x.split()))
# Применяем функцию remove_stopwords ко всем значениям в столбце dealer_name
dealer_name = dealer_name.apply(lambda x: remove_stopwords(x))
# Удаляем последовательности цифр, начинающиеся с пробела, из каждого значения в столбце dealer_name
dealer_name = dealer_name.apply(lambda x: re.sub('\w*\d\w*', '', x))
# Убираем лишние пробелы в каждом значении столбца dealer_name
dealer_name = dealer_name.apply(lambda x: ' '.join(x.split()))

In [ ]:
# Объединение столбцов dealer_name и dealer_volume в новый столбец dealer_corpus
dealer_corpus = dealer_name + ' ' + dealer_volume
dealer_corpus

In [118]:
# Получаем новые векторные представления
corpus_embeddings_new = model.encode(product_corpus.values, convert_to_tensor=True)

In [119]:
# Вызов функции recommendations_for_metric для оценки модели по метрике
empty_result_newf, false_newf, scores_newf = recommendations_for_metric(model, dealer_corpus, corpus_embeddings_new, k=5)

  3%|▎         | 513/20416 [01:15<43:06,  7.70it/s]<ipython-input-80-c9a29b773efa>:12: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not true_id:
100%|██████████| 20416/20416 [41:35<00:00,  8.18it/s]


In [122]:
accuracy_k(scores_newf)

0.95